In [1]:
import cPickle as pickle
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import *
import numpy as np
import sys
import os.path
sys.path.append(os.path.join('', 'TFIDF'))
sys.path.append(os.path.join('', 'util'))
from timer import Timer

In [ ]:
from HTMLParser import HTMLParser

class MLStripper(HTMLParser):
    def __init__(self):
        self.reset()
        self.fed = []
    def handle_data(self, d):
        self.fed.append(d)
    def get_data(self):
        return ''.join(self.fed)

def strip_tags(html):
    s = MLStripper()
    s.feed(html)
    return s.get_data()

In [ ]:
questions = pickle.load(open("lstOfQuestions.pkl", "rb"))
questions = questions[:40000]

words_dict = {}
question_texts = []
total = len(questions)
count = 0
for question in questions :
    if count % (total / 10) == 0:
        print count
    VALID_TAGS = ['p']

    soup = BeautifulSoup(question['body'])

    VALID_TAGS = ['p']
    INVALID_TAGS = ['code', 'a']

    text = ' '.join([s.get_text() for s in soup('p')])

    tokens = nltk.word_tokenize(text)

    stemmer = PorterStemmer()

    words = [stemmer.stem(token.encode('ascii', 'ignore').lower()) for token in tokens]

    question_texts.append(words)
    
    for word in words :
        if word in words_dict :
            words_dict[word] += 1
        else :
            words_dict[word] = 1
    
    count += 1
    
id_hash = {}
count = 0
for question in questions:
    if question['question_id'] not in id_hash:
        id_hash[question['question_id']] = 1
        count += 1

print count

# Go through dictionary, removing entries that have a value less
# than delta
delta = 2
words_dict2 = {key: value for key, value in words_dict.items()
                 if value >= delta }

word_indices = {}
index = 0
for word in words_dict2 :
    word_indices[word] = index
    index += 1

word_indices = list(word_indices.iteritems())

output = open('wordIndices_md.pkl', 'wb')
pickle.dump(word_indices, output, -1)
output.close()

output = open('questionTexts_md.pkl', 'wb')
pickle.dump(question_texts, output, -1)
output.close()

In [2]:
try:
     tfidf = reload(tfidf)
except NameError:
    import TFIDF_numpy as tfidf

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))
    
tfidf.init_globals()
tfidf.load_questions(question_texts)
tfidf.load_indices(word_indices)
tfidf.init_tfs()
serial_tfs = tfidf.create_tfs()
serial_idf = tfidf.calculate_idf()
serial_tfidfs = tfidf.calculate_tfidfs()
tfidf.calculate_tfidf_norms()
# tfidf.calculate_cossim(question_texts[0])

Time for initialization: 5.00679016113e-06
Time for load_questions: 9.53674316406e-07
Time for load_indices: 0.0154359340668
Time for init_tfs: 3.5346660614
Time for create_tfs: 2.24818706512
Time for create_idf: 2.10253405571
Time for calculate_tfidfs: 2.9298620224
Time for calculate_tfidf_norms: 9.59677505493


In [3]:
try :
    tfidf_c = reload(tfidf_c)
except :
    import TFIDF_cython as tfidf_c

try :
    word_indices
except NameError:
    word_indices = pickle.load(open('wordIndices_sm.pkl', 'rb'))

try :
    question_texts
except NameError:
    question_texts = pickle.load(open('questionTexts_sm.pkl', 'rb'))

# Preprocess for cython code
tfidf_c.init_globals(4)
tfidf_c.load_questions(question_texts)
tfidf_c.load_indices(word_indices)
tfidf_c.init_tfs()
cython_tfs = tfidf_c.create_tfs()
cython_idf = tfidf_c.calculate_idf()
tfidf_c.init_tfidfs()
cython_tfidfs = tfidf_c.calculate_tfidfs()
# tfidf.calculate_cossim(question_texts[0])

Compiling with /usr/local/bin/clang-omp
Time for Initialization: 1.00135803223e-05
Time for load_questions: 0.412717103958
Time for load_indices: 0.0449950695038
Time for init_tfs: 3.33657503128
Time for create_tfs: 0.179300785065
Time for calculate_idf: 0.269981145859
Time for int_tfidfs: 2.42379283905
Time for calculate_tfidfs: 0.20812702179


In [ ]:
# Verification
print np.linalg.norm(np.asarray(cython_tfs) - serial_tfs)
print np.linalg.norm(np.asarray(cython_idf) - serial_idf)
print np.linalg.norm(np.asarray(cython_tfidfs) - serial_tfidfs)

In [ ]:
np.asarray(cython_tfs).index(983.57928)

In [ ]:
np.where(np.asarray(cython_tfs) == 983.57928)

In [ ]:
serial_tfs[6378][18644]

In [ ]:
for (key, value) in word_indices :
    if value == 18644 :
        print key

In [ ]:
np.sum(serial_idf - np.asarray(cython_idf))

In [ ]:
dir(tfidf_c)

In [ ]:
with Timer() as t :
    a = list(word_indices.iteritems())
print t.interval
with Timer() as t :
    b = dict(a)
print t.interval

In [ ]:
import xxhash


In [ ]:
simhashes = []
for u in range(len(question_texts)):
    W = np.zeros(64)
    for i in range(len(question_texts[u])):
#         print (question_texts[u][i])
        try:
            wordhash = xxhash.xxh64(question_texts[u][i]).intdigest()
            counter = 63
            while wordhash > 0:
                bit = wordhash % 2
                if bit:
                    W[counter] += tfidf_vectors[u][i]
                else:
                    W[counter] -= tfidf_vectors[u][i]
                wordhash = wordhash >> 1
                counter -= 1
        except:
            pass
    simhash = 0
    for i in range(len(W)):
        if W[i] >= 0:
            simhash += 1
        if i < len(W)-1:
            simhash = simhash << 1
    simhashes.append(simhash)

In [ ]:
simhashes[10]




In [ ]:
for i in xrange(len(simhashes)):
    for j in xrange(i, len(simhashes)):
        if i != j:
            

In [ ]:
numpy_questions

In [ ]:
np.array(list(word_indices.iteritems()))

In [ ]:
a = np.array(list(word_indices.iteritems()))
a[:,0]

In [ ]:
word_indices = pickle.load(open('wordIndices.pkl', 'rb'))

In [ ]:
len(word_indices)

In [ ]:
"%".encode('utf-8')